In [ ]:
import os
import re 
import numpy as np 
import pandas as pd 
import anndata as ad 
import scanpy as sc 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
## read data and map cell type annotations to Leiden clusters for each panel 
panels = ['immune','tumor','tracked']
panel_df_dict = {}

for p in panels:

    # read annotations and turn into a dictionary with leiden clusters as keys 
    annotation_df = pd.read_csv(f'annotations/E06_{p}_annotations.csv')
    annotation_map = pd.Series(annotation_df.cell_type_refined.values, index = annotation_df.cluster).to_dict()

    # read leiden clustered data
    df = pd.read_csv(f'data/E06_{p}_indiv_leiden.csv')

    # map cell types using the annotation dict 
    df[f'cell_type_{p}'] = df['leiden'].map(annotation_map)

    # save df in dict with panel as key
    panel_df_dict[p] = df

In [ ]:
## merge immune and tumor cell types into the tracked dataframe 
# prep for merging with tracked data
immune = panel_df_dict['immune'][['CellID','cell_type_immune']]
tumor = panel_df_dict['tumor'][['CellID','cell_type_tumor']]

# merge cluster assignments across panels 
E06_tracked = pd.merge(panel_df_dict['tracked'], immune, left_on = 'immune_CellID', right_on = 'CellID')
E06_tracked = pd.merge(E06_tracked, tumor, left_on = 'tumor_CellID', right_on = 'CellID')

E06_immune = panel_df_dict['E06']['immune']
E06_tumor = panel_df_dict['E06']['tumor']

switched_cells_tracked = E06_tracked[(E06_tracked.cell_type_tumor == 'aSMA+ Stroma') &
            (E06_tracked.cell_type_tracked == 'Macrophages') &
            (E06_tracked.cell_type_immune == 'Macrophages')]


switched_cells_immune = E06_immune[E06_immune['CellID'].isin(list(switched_cells_tracked.immune_CellID))]
switched_cells_tumor = E06_tumor[E06_tumor['CellID'].isin(list(switched_cells_tracked.tumor_CellID))]

In [ ]:
print(len(switched_cells_tracked), len(switched_cells_immune), len(switched_cells_tumor))

In [ ]:
adata = ad.AnnData(X = E06_tumor[tumor_markers], obs = E06_tumor[['CellID']])
adata.obs['switched_ID'] = np.where(adata.obs.CellID.isin(list(switched_cells_tracked.tumor_CellID)), 'switched', 'other')
sc.tl.rank_genes_groups(adata, groupby='switched_ID')
tumor_switch_ranks = sc.get.rank_genes_groups_df(adata, group=['switched'])
# sc.pl.rank_genes_groups(adata, groups = ['switched'], save = 'tumor_switch.png')

In [ ]:
# fig,ax = plt.subplots(1,2)

adata = ad.AnnData(X = E06_immune[immune_markers], obs = E06_immune[['CellID']])
adata.obs['switched_ID'] = np.where(adata.obs.CellID.isin(list(switched_cells_tracked.immune_CellID)), 'switched', 'other')
sc.tl.rank_genes_groups(adata, groupby='switched_ID')
immune_switch_ranks = sc.get.rank_genes_groups_df(adata, group=['switched'])
# fig = sc.pl.rank_genes_groups(adata, groups = ['switched'],save = 'immune_switch.png')

In [ ]:
tracked_markers = ['tumor_CD3', 'tumor_Vimentin', 'tumor_aSMA',
 'tumor_Ecad', 'tumor_Ki67', 'tumor_CD45', 'tumor_CK14',
 'tumor_CK19', 'tumor_CK17', 'tumor_PCNA', 
 'tumor_PanCK', 'tumor_CD31', 'immune_CD3', 
 'immune_GRZB', 'immune_Ki67', 'immune_PanCK', 
 'immune_CD45', 'immune_CD68', 'immune_CD3d', 
 'immune_CD8a', 'immune_CD163','immune_aSMA', 
 'immune_CD14', 'immune_CD4', 'immune_FOXP3',
 'immune_CD11b','immune_CD20','immune_MHC_II_DPB1']

E06_tracked = E06_tracked.reset_index()
adata = ad.AnnData(X = E06_tracked[tracked_markers], obs = E06_tracked[['CellID_x']])
adata.obs['switched_ID'] = np.where(adata.obs.CellID_x.isin(list(switched_cells_tracked.CellID_x)), 'switched', 'other')
sc.tl.rank_genes_groups(adata, groupby='switched_ID')
tracked_switch_ranks = sc.get.rank_genes_groups_df(adata, group=['switched'])
# sc.pl.rank_genes_groups(adata, groups = ['switched'], save = 'tracked_switch.png')

In [ ]:
tumor_switch_ranks['panel'] = 'Tumor'
immune_switch_ranks['panel'] = 'Immune'
tracked_switch_ranks['panel'] = 'Combined'

tumor_switch_ranks['rank'] = tumor_switch_ranks.index
immune_switch_ranks['rank'] = immune_switch_ranks.index
tracked_switch_ranks['rank'] = tracked_switch_ranks.index

tumor_switch_ranks['marker_origin'] = 'tumor'
immune_switch_ranks['marker_origin'] = 'immune'
tracked_switch_ranks['marker_origin'] = [re.split('_', x)[0] for x in list(tracked_switch_ranks['names'])]
tracked_switch_ranks['names'] = [re.split('_', x)[1] for x in list(tracked_switch_ranks['names'])]

merged = pd.concat([tumor_switch_ranks, immune_switch_ranks, tracked_switch_ranks])

merged['names'] = merged['names'].replace('MHC','MHC_II_DPB1')

In [ ]:
merged.to_csv('E06_aSMA_Macrophage_ID_switch_ranks.csv', index = False)

In [ ]:
merged